In [2]:
pip install pandas

  Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl (11.0 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ---------------------------------------- 0/3 [pytz]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   ------------- -------------------------- 1/3 [tzdata]
   -------------------------- ------------- 2/3 [pandas]
   -------------------------- ------------- 2/3 [pandas]
   -

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import DBSCAN
from sklearn.metrics import accuracy_score , classification_report
from sentence_transformers import SentenceTransformer
import re

c:\Users\Patel\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
print(os.getcwd())
import os
print(os.listdir())  # Lists all files in the current directory



c:\Users\Patel\OneDrive\Desktop\pl\log classify
['.venv', 'classify.py', 'Data', 'model', 'requirements.txt', 'synthetic_logs.csv', 'train.ipynb', 'Training']


In [3]:
df=pd.read_csv('synthetic_logs.csv')

In [4]:
df.head(20)

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert


In [5]:
print(df.columns)

Index(['timestamp', 'source', 'log_message', 'target_label', 'complexity'], dtype='object')


In [6]:
df['source'].value_counts()

source
ThirdPartyAPI      496
ModernHR           492
BillingSystem      479
AnalyticsEngine    471
ModernCRM          465
LegacyCRM            7
Name: count, dtype: int64

In [7]:
# the lowest logs belong to LegacyCRM source the rest looks ok the first step is to do the clusturing of the logs 
# and create regx expressions for those categories of errors

In [8]:
X=df['log_message']

In [9]:
X

0       nova.osapi_compute.wsgi.server [req-b9718cd8-f...
1          Email service experiencing issues with sending
2               Unauthorized access to data was attempted
3       nova.osapi_compute.wsgi.server [req-4895c258-b...
4       nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...
                              ...                        
2405    nova.osapi_compute.wsgi.server [req-96c3ec98-2...
2406    User 3844 account experienced multiple failed ...
2407    nova.metadata.wsgi.server [req-b6d4a270-accb-4...
2408        Email service affected by failed transmission
2409    Repeated failed login attempts occurred for us...
Name: log_message, Length: 2410, dtype: object

In [10]:
transformers=SentenceTransformer('all-MiniLM-L6-v2')
embeddings=transformers.encode(X)

In [11]:
embeddings.shape

(2410, 384)

In [12]:
# The embeddings for log_message is created 

In [13]:
clusters=DBSCAN(eps=0.2,min_samples=1,metric='cosine').fit(embeddings)


In [14]:
df['clusters']=clusters.labels_

In [15]:
df['clusters'].value_counts().head(10)

clusters
0     1017
5      147
11     100
13      86
7       60
8       60
21      58
3       57
4       53
17      52
Name: count, dtype: int64

In [16]:
c1=df[df['clusters']==0]
c2=df[df['clusters']==5]
c3=df[df['clusters']==11]
c4=df[df['clusters']==13]
c6=df[df['clusters']==8]
c7=df[df['clusters']==21]
c9=df[df['clusters']==4]

In [17]:
c1['log_message'].str[:20].unique()

array(['nova.osapi_compute.w', 'nova.metadata.wsgi.s'], dtype=object)

In [18]:
c1['log_message'].unique()[1]

'nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180'

In [19]:
c1['log_message'].unique()

array(['nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118',
       'nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180',
       'nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791',
       ...,
       'nova.osapi_compute.wsgi.server [req-2c9c783f-3c7a-4844-87b1-e207a38c74ab 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10

In [20]:
def regx_creator(msg):
    patterns={
        r"nova\.(osapi|metadata).*":'HTTP Status',
        r"nova\.compute.*":'Resource Usage',
        r"User User\d+ logged (out|in).*":'user action',
        r"Backup (started|ended) at.*":'System Notification',
        r"Backup completed successfully.":'System Notification',
        r"System updated to version.*":'System Notification',
        r"File data_\d+\.csv uploaded successfully by user User.*":'System Notification'}
    
    for pattern,value in patterns.items():
        if re.search(pattern,msg):
            return value
    return None

In [21]:
print(re.search(r"File data_\d+\.csv uploaded successfully by user User.*",'File data_6169.csv uploaded successfully by user User953.'))

<re.Match object; span=(0, 57), match='File data_6169.csv uploaded successfully by user >


In [22]:
print(regx_creator('nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180'))

HTTP Status


In [23]:
df['clusters']=df['log_message'].apply(regx_creator)

In [24]:
unidentified_df=df[df['clusters'].isna()]

In [25]:
unidentified_df['source'].value_counts()

source
ModernHR           179
BillingSystem      173
AnalyticsEngine    172
ThirdPartyAPI      168
ModernCRM          160
LegacyCRM            7
Name: count, dtype: int64

In [26]:
LegacyCRM_df=unidentified_df[unidentified_df['source']=='LegacyCRM']

In [27]:
LegacyCRM_df

,timestamp,source,log_message,target_label,complexity,clusters
60,2025-10-06 16:55:23,LegacyCRM,Lead conversion failed for prospect ID 7842 du...,Workflow Error,llm,None
255,2025-05-03 16:55:35,LegacyCRM,API endpoint 'getCustomerDetails' is deprecate...,Deprecation Warning,llm,None
377,2025-06-24 12:16:29,LegacyCRM,Customer follow-up process for lead ID 5621 fa...,Workflow Error,llm,None
1325,2025-04-17 07:33:44,LegacyCRM,Escalation rule execution failed for ticket ID...,Workflow Error,llm,None
1734,2025-04-30 07:47:30,LegacyCRM,The 'ExportToCSV' feature is outdated. Please ...,Deprecation Warning,llm,None
1826,2025-01-23 10:33:36,LegacyCRM,Support for legacy authentication methods will...,Deprecation Warning,llm,None
2217,2025-05-12 09:46:54,LegacyCRM,Task assignment for TeamID 3425 could not comp...,Workflow Error,llm,None


In [28]:

unknown_df=unidentified_df[unidentified_df['source']!='LegacyCRM']

In [29]:
unknown_df.to_csv("Data/unknown_df.csv")
LegacyCRM_df.to_csv("Data/LegacyCRM_df.csv")

In [30]:
#using LegacyCRM source for LLM classification as there are only 7 of them and only 2 categories

In [31]:
unknown_df['log_message']

1          Email service experiencing issues with sending
2               Unauthorized access to data was attempted
6               Shard 6 replication task ended in failure
10               Email server encountered a sending fault
11        Critical system unit error: unit ID Component55
                              ...                        
2398    Potential vulnerability exploit detected from ...
2402              Replication error occurred for shard 10
2406    User 3844 account experienced multiple failed ...
2408        Email service affected by failed transmission
2409    Repeated failed login attempts occurred for us...
Name: log_message, Length: 852, dtype: object

In [32]:
encoder=SentenceTransformer('all-MiniLM-L6-v2')
X = encoder.encode(unknown_df['log_message'].astype(str).tolist())


In [33]:
from sklearn.preprocessing import LabelEncoder

In [34]:
labels=LabelEncoder()
y=labels.fit_transform(unknown_df['target_label'])

In [35]:
y.shape

(852,)

In [36]:
X.shape

(852, 384)

In [37]:
lg=LogisticRegression()

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [40]:
print(X_test.shape,y_test.shape)

(171, 384) (171,)


In [45]:
y_pred

array([0, 0, 1, 3, 0, 2, 3, 1, 1, 1, 2, 2, 2, 0, 3, 2, 1, 1, 2, 2, 4, 4,
       2, 0, 2, 2, 1, 0, 2, 2, 0, 2, 1, 3, 0, 1, 1, 2, 1, 1, 0, 2, 1, 0,
       0, 0, 0, 2, 4, 2, 3, 2, 1, 2, 1, 2, 2, 4, 3, 3, 1, 2, 0, 3, 1, 0,
       1, 1, 1, 2, 0, 1, 1, 1, 2, 2, 3, 1, 0, 2, 2, 1, 0, 1, 0, 2, 4, 0,
       3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2,
       2, 2, 0, 2, 1, 2, 0, 1, 3, 2, 0, 2, 3, 2, 2, 1, 1, 2, 3, 1, 0, 0,
       2, 1, 0, 1, 3, 2, 1, 0, 1, 0, 3, 3, 4, 0, 3, 2, 2, 0, 1, 3, 3, 2,
       2, 1, 1, 1, 0, 1, 2, 0, 0, 0, 3, 0, 2, 2, 2, 4, 2])

In [41]:
lg.fit(X_train,y_train)

LogisticRegression()

In [42]:
y_pred=lg.predict(X_test)

In [46]:
labels.classes_

array(['Critical Error', 'Error', 'Security Alert', 'System Notification',
       'User Action'], dtype=object)

In [43]:
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error


In [ ]:
y_pred.shape
y_test.shape

(171,)

In [43]:
print(accuracy_score(y_pred,y_test))

0.9766081871345029


In [44]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96        40
           1       0.93      0.97      0.95        40
           2       0.98      1.00      0.99        62
           3       1.00      1.00      1.00        22
           4       1.00      1.00      1.00         7

    accuracy                           0.98       171
   macro avg       0.98      0.98      0.98       171
weighted avg       0.98      0.98      0.98       171



In [45]:
import joblib

In [86]:
model=joblib.dump(lg,'model/linear_regression.joblib')